In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!sudo pip install gmr

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [1]:
import pandas as pd
import os
import csv
import math
import datetime as dt

basedir = '/content/drive/My Drive/MSCodeGoogleColab/'

In [0]:


def preprocess_data(filename):
    inputfile = basedir + filename
    data = pd.read_csv(inputfile)

    data.iloc[:, 4] = pd.to_datetime(data.iloc[:, 4])
    timeSlots = data.iloc[:, 4]
    username = data.iloc[:, 0].iloc[0]

    binname = basedir+'/processeddata/processed_1' + str(filename)


    if not os.path.exists(os.path.dirname(binname)):
        try:
            os.makedirs(os.path.dirname(binname))
        except OSError as exc:
            print(exc.errno)



    rowlist = []

    with open(binname, "w") as outputFile:
        csvWriter = csv.writer(outputFile)

        row_iterator = data.iterrows()
        _, last = next(row_iterator)  # take first item from row_iterator

        for i, row in row_iterator:
            # last = row

            previous = last  # datetime.strptime(p, '%Y-%m-%d %H:%M:%S')  # "2015-09-03 16:25:45"
            current = row  # datetime.strptime(c, '%Y-%m-%d %H:%M:%S')

            print('previous: ', previous.iloc[4])
            print('current: ', current.iloc[4])

            sbp = previous.iloc[1]  # data['systolic'][i - 1]
            dbp = previous.iloc[2]  # data['diastolic'][i - 1]
            hr = previous.iloc[3]  # data['pulse'][i - 1]

            csvWriter.writerow([username, sbp, dbp, hr, previous.iloc[4]])

            delta = current.iloc[4] - previous.iloc[4]
            days = delta.days
            hours = math.floor((delta.seconds) / 3600)
            mins = math.floor(delta.seconds / 60)


            if hours > 1:
                print('Found missing value at index:', i)

                new_date = previous.iloc[4] + datetime.timedelta(hours=1)

                reached = 0

                while reached == 0:

                    if new_date < current.iloc[4]:
                        # create new_date entry in file
                        csvWriter.writerow([username, 0, 0, 0, new_date])
                        new_date = new_date + datetime.timedelta(hours=1)
                    else:
                        # csvWriter.writerow([smoking, gender, height, 0, 0, 0, current['recordDateTime'], weight, age])
                        reached = 1
                        last = row
            else:
                 last = row

        outputFile.close()



In [0]:
def preprocess_etiqa_data(filename):
    filename = 'Etiqa_ajanthini'
    inputfile = basedir+'/'+filename+'.csv'
    missingdata = 0               #keeps track of how many missing entries in sample data
    
    data = pd.read_csv(inputfile)
    ps = pd.to_datetime(data.iloc[:, 3])
    data.iloc[:, 3] = ps
    timeSlots = data.iloc[:, 3]
    username = filename
    #print(data.iloc[:3])

    outputFileName = basedir+'/first_processeddata/processed_'+filename+'.csv'

    if not os.path.exists(os.path.dirname(outputFileName)):
        try:
            os.makedirs(os.path.dirname(outputFileName))
        except OSError as exc:
            print (exc.errno)

    with open(outputFileName, "w") as outputFile:
        csvWriter = csv.writer(outputFile)

        row_iterator = data.iterrows()
        
        _, last = row_iterator.next()  # take first item from row_iterator

        for i, row in row_iterator:

            previous = last  # datetime.strptime(p, '%Y-%m-%d %H:%M:%S')  # "2015-09-03 16:25:45"
            current = row  # datetime.strptime(c, '%Y-%m-%d %H:%M:%S')

            #print('previous: ', previous.iloc[3])
            #print('current: ', current.iloc[3])

            sbp = previous.iloc[0]  # data['systolic'][i - 1]
            dbp = previous.iloc[1]  # data['diastolic'][i - 1]
            hr = previous.iloc[2]  # data['pulse'][i - 1]

            csvWriter.writerow([sbp, dbp, hr, previous.iloc[3]])

            delta = current.iloc[3] - previous.iloc[3]
            days = delta.days
            hours = math.floor((delta.seconds) / 3600)
            mins = math.floor(delta.seconds / 60)

            loopstop = current.iloc[3]  #comparison to stop loop of adding missing values

            if days > 0 or hours > 1:
                print('Found missing value at: ', previous.iloc[3].isoformat(' '))
               
                if days > 0:
                    #only increment time for the current date so as to avoid entire zero dates
                    prevDate = previous.iloc[3]
                    print('prev date:', prevDate.isoformat(' '))
                 
                    loopstop = dt.datetime(year= prevDate.year, month=prevDate.month, day=prevDate.day, hour=23, minute=0, second=0)
                    print('loopstop:', loopstop.isoformat(' '))

                reached = 0
                new_date = previous.iloc[3] + dt.timedelta(hours=1)
                while reached == 0:
                    if new_date < loopstop:
                        # create new_date entry in file
                        csvWriter.writerow([0, 0, 0, new_date])
                        missingdata += 1
                        new_date = new_date + dt.timedelta(hours=1)
                    else:
                        reached = 1
                        last = row
                        print('last created entry at: ', new_date.isoformat(' '))
                        
                        if new_date == loopstop:
                          print('all entries entered ok')
                          
                        else:
                          print('add last loopstop entry')
                          csvWriter.writerow([0, 0, 0, loopstop])
                          missingdata += 1
                        
                        #check if date for this cycle starts at 0 hrs, else generate missing entries
                        cycledate = last.iloc[3]
                        
                        if cycledate.hour > 0 and (cycledate.day > new_date.day):
                          #generate missing values untill cycle date
                          
                          cyclestart = dt.datetime(year= cycledate.year, month=cycledate.month, day=cycledate.day, hour=0, minute=0, second=0)
                          print('add missing values in cycle from: ', cyclestart.isoformat(' '))
                          
                          while cyclestart < cycledate:
                            csvWriter.writerow([0,0,0, cyclestart])
                            missingdata += 1
                            cyclestart = cyclestart + dt.timedelta(hours=1)
                            
                        else:
                          print('normal cycle begin')
            else:
              last = row
             

        outputFile.close()
        
        totalsamplesize = len(data) + missingdata
        print('****************************')
        print(totalsamplesize)
        print(missingdata)
        return missingdata/totalsamplesize


In [0]:


#preprocess_data('etiqa_azim.csv')

missingdata = preprocess_etiqa_data('dd')
print('****************************')
print('total missing in file: '+str(missingdata))